In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

TensorFlow 2.x selected.
Version:  2.0.0-rc0
Eager mode:  True
Hub version:  0.6.0
GPU is available


In [2]:
train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], 
                                  batch_size=-1, as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

# /Users/jheaton/tensorflow_datasets/imdb_reviews/plain_text/0.1.0

    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


%% **Loading the pretrained Google "gnews" model which will convert the raw data into 20 dimensional vector **

In [0]:
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], 
                           dtype=tf.string, trainable=True)

In [4]:
# Raw training data of the movie review  
train_examples[:3]

array([b"This was soul-provoking! I am an Iranian, and living in th 21st century, I didn't know that such big tribes have been living in such conditions at the time of my grandfather!<br /><br />You see that today, or even in 1925, on one side of the world a lady or a baby could have everything served for him or her clean and on-demand, but here 80 years ago, people ventured their life to go to somewhere with more grass. It's really interesting that these Persians bear those difficulties to find pasture for their sheep, but they lose many the sheep on their way.<br /><br />I praise the Americans who accompanied this tribe, they were as tough as Bakhtiari people.",
       b'I have no idea what the other reviewer is talking about- this was a wonderful movie, and created a sense of the era that feels like time travel. The characters are truly young, Mary is a strong match for Byron, Claire is juvenile and a tad annoying, Polidori is a convincing beaten-down sycophant... all are beautiful,

In [5]:
# Converting the raw text files into 20 dimensional vetors
hub_layer(train_examples[:3])

<tf.Tensor: id=313, shape=(3, 20), dtype=float32, numpy=
array([[ 2.2488396 , -1.4537774 ,  1.97866   , -0.73290443, -2.2218246 ,
        -4.054772  , -1.6585274 ,  1.8872426 ,  1.8315402 ,  0.45302168,
        -0.50105846,  1.3690453 , -1.9862492 ,  0.4043505 , -5.337918  ,
         1.5515825 ,  3.6844683 , -3.3751655 , -3.4578393 , -1.1764543 ],
       [ 3.0298512 , -3.1155329 ,  3.231569  , -2.470162  , -3.6778672 ,
        -2.7249148 , -1.5884265 ,  0.8121041 ,  4.3448696 , -1.4254425 ,
        -1.9567002 ,  0.7446213 , -0.9539236 ,  0.4037092 , -4.982661  ,
         0.9707939 ,  3.717819  , -1.3615017 , -2.881222  , -1.2961531 ],
       [ 3.041689  , -4.7446456 ,  4.733248  ,  0.30785134, -6.9846196 ,
        -5.9006767 , -4.4500394 ,  2.3718796 ,  5.0403566 ,  0.4119934 ,
        -2.8661985 ,  0.6986558 ,  0.5313177 ,  1.4525845 , -7.200312  ,
         2.9011002 ,  6.392641  , -2.6039283 , -2.9440727 , -2.4246173 ]],
      dtype=float32)>

In [6]:
# Build the network with the hub layer and adding 16 hidden layer and one output layer since binary classification
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Compile network 
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
# Split the data into 10000 samples for validation and remaining 15000 samples for train set
x_val = train_examples[:10000]
partial_x_train = train_examples[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [9]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Train on 15000 samples, validate on 10000 samples
Epoch 1/40
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


15000/15000 [==============================] - 3s 227us/sample - loss: 1.1375 - accuracy: 0.5123 - val_loss: 0.7772 - val_accuracy: 0.5691
Epoch 2/40
15000/15000 [==============================] - 3s 169us/sample - loss: 0.7444 - accuracy: 0.5630 - val_loss: 0.6908 - val_accuracy: 0.5897
Epoch 3/40
15000/15000 [==============================] - 3s 171us/sample - loss: 0.6806 - accuracy: 0.6013 - val_loss: 0.6508 - val_accuracy: 0.6270
Epoch 4/40
15000/15000 [==============================] - 3s 169us/sample - loss: 0.6425 - accuracy: 0.6361 - val_loss: 0.6215 - val_accuracy: 0.6574
Epoch 5/40
15000/15000 [==============================] - 3s 170us/sample - loss: 0.6108 - accuracy: 0.6663 - val_loss: 0.5955 - val_accuracy: 0.6777
Epoch 6/40
15000/15000 [==============================] - 3s 169us/sample - loss: 0.5807 - accuracy: 0.6958 - val_loss: 0.5706 - val_accuracy: 0.7042
Epoch 7/40
15000/15000 [==============================] - 3s 170us/sample - loss: 0.5516 - accuracy: 0.7233 - v

In [10]:
# Evaluate the model 
results = model.evaluate(test_data, test_labels, verbose=0)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.368
accuracy: 0.861
